<a href="https://colab.research.google.com/github/Niv0902/Shablool/blob/main/ShabloolSearchEngineHM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1**
pip
imports
firebase url
stemmer
stopwords

In [1]:


!pip install firebase paho-mqtt nltk ipywidgets --quiet
!pip install ipywidgets
!pip install paho-mqtt firebase-admin gradio


import json
import time
import re
import threading
import paho.mqtt.client as mqtt
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from firebase import firebase as fb
from collections import defaultdict
from nltk.stem import PorterStemmer
import ipywidgets as widgets
from IPython.display import display, clear_output,HTML
import pandas as pd
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import re





# התחברות לפיירבייס
FIREBASE_URL = 'https://shabloolsearchengine-default-rtdb.asia-southeast1.firebasedatabase.app/'
FBconn = fb.FirebaseApplication(FIREBASE_URL, None)
print('✅ Connected to Firebase')

# הגדרת stemmer ו־stop words
stemmer = PorterStemmer()
custom_stop_words = {
    'a', 'an', 'the', 'and', 'or', 'but',
    'to', 'from', 'of', 'in', 'on', 'at', 'for', 'by', 'with', 'as',
    'this', 'that', 'these', 'those', 'it', 'its',
    'be', 'is', 'are', 'was', 'were', 'has', 'have', 'had',
    'not', 'no', 'yes', 'true', 'false', 'null', 'none'
}
loggedin_user = None # Global variable to hold currently logged in user
loggedin_user_rank = None # Global variable to hold currently logged in users rank

✅ Connected to Firebase


**2**
Search function

In [2]:
def search_mqtt(query, num_results=5):
    index = FBconn.get('/search_index', None)
    titles = FBconn.get('/doc_titles', None)
    urls = FBconn.get('/doc_urls', None)

    if not index or not titles or not urls:
        return []

    query_words = [stemmer.stem(w.lower()) for w in re.findall(r'\w+', query)]
    if not query_words:
        return []

    page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})

    for term in query_words:
        if term in index:
            doc_freqs = index[term]['DocIDs']  # <- דיקט של {doc_id: freq}
            for doc_id, freq in doc_freqs.items():
                page_scores[doc_id]['matches'] += 1
                page_scores[doc_id]['total_freq'] += freq

    ranked = sorted([
        (doc_id, score['matches'], score['total_freq'])
        for doc_id, score in page_scores.items()
    ], key=lambda x: (x[1], x[2]), reverse=True)

    results = []
    for doc_id, matches, total_freq in ranked[:num_results]:
        title = titles.get(doc_id, "Unknown Title")
        url = urls.get(doc_id, "Unknown URL")
        results.append({
            'doc_id': doc_id,
            'title': title,
            'url': url,
            'matching_words': matches,
            'total_frequency': total_freq
        })

    return results


**3**


*   Fetch Url pages from site
*   Using Beautiful soap
*   List item
*   List item




In [3]:
base_url = "https://mqtt.org"
start_page = base_url + "/"

from collections import defaultdict

# משתמשים בספירת תדירויות אמיתית!
term_counts = defaultdict(lambda: defaultdict(int))
doc_titles = {}
doc_urls = {}

def clean_and_tokenize(text):
    words = re.findall(r'\b[a-zA-Z]{2,}\b', text.lower())
    return [stemmer.stem(word) for word in words if word not in custom_stop_words]

# שלב 1: סריקת קישורים מהעמוד הראשי
try:
    response = requests.get(start_page)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a', href=True)

    pages = {}
    page_id = 1
    for link in links:
        href = link['href']
        if href.startswith('/'):
            full_url = base_url + href
            if full_url not in pages.values():
                doc_id = f"page{page_id}"
                pages[doc_id] = full_url
                doc_urls[doc_id] = full_url
                page_id += 1
except Exception as e:
    print(f"❌ Failed to crawl links: {e}")
    pages = {}

# שלב 2: אינדוקס וספירה
for doc_id, url in pages.items():
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text()
        words = clean_and_tokenize(text)

        for word in words:
            term_counts[word][doc_id] += 1

        title = soup.title.string.strip() if soup.title and soup.title.string else url
        doc_titles[doc_id] = title

        print(f"✅ Indexed {doc_id}: {title}")
    except Exception as e:
        print(f"❌ Failed to load {url}: {e}")

# שלב 3: הכנה להעלאה
firebase_ready_index = {
    term: {"term": term, "DocIDs": dict(doc_map)}
    for term, doc_map in term_counts.items()
}

FBconn.put('/', 'search_index', firebase_ready_index)
FBconn.put('/', 'doc_titles', doc_titles)
FBconn.put('/', 'doc_urls', doc_urls)

print("✅ All data uploaded to Firebase.")


✅ Indexed page1: Getting started
✅ Indexed page2: MQTT Specification
✅ Indexed page3: Software
✅ Indexed page4: Use Cases
✅ Indexed page5: FAQ
✅ Indexed page6: Use Cases
✅ Indexed page7: Use Cases
✅ Indexed page8: Use Cases
✅ Indexed page9: Use Cases
✅ Indexed page10: Use Cases
✅ Indexed page11: Use Cases
✅ Indexed page12: Legal
✅ All data uploaded to Firebase.


**4**

*   Read sensor data
*   Process sensor message to fire base


In [4]:

# שמירת ההודעה לפיירבייס לפי topic
def process_message(data, source):
    timestamp = str(int(time.time() * 1000))
    FBconn.put(f'/sensor_data/{source}', timestamp, data)

# עיבוד הודעות נכנסות
def on_message(client, userdata, msg):
    try:
        payload = json.loads(msg.payload.decode())
        topic_name = msg.topic.split("/")[-1]  # indoor / outdoor
        process_message(payload, topic_name)
        print(f"✅ Saved sensor data ({topic_name}): {payload}")
    except json.JSONDecodeError:
        print("❌ Invalid JSON received")

# התחברות ותחילת הקשבה
def start_mqtt():
    client = mqtt.Client()
    client.on_message = on_message
    client.connect("test.mosquitto.org", 1883, 60)

    # נרשם לשני המקורות
    client.subscribe("braude/D106/indoor")
    client.subscribe("braude/D106/outdoor")

    client.loop_start()
    print("📡 Listening for sensor data (MQTT)...")

    # האזנה לדקה (או לשנות לכמה שתרצה)
    time.sleep(60)

    client.loop_stop()
    print("✅ Finished listening")


In [5]:
start_mqtt()

<ipython-input-4-c46663234ee1>:18: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


📡 Listening for sensor data (MQTT)...
✅ Finished listening


In [6]:
def logout_user():
    global loggedin_user
    global loggedin_user_rank
    loggedin_user = None
    loggedin_user_rank = None
    print("Logged out.")
    main_dashboard()

def employee_screen():
  with content_output:
    content_output.clear_output()

In [7]:
def user_screen():
    with content_output:
        content_output.clear_output()
        global loggedin_user
        global loggedin_user_rank

        # אם כבר מחובר, נווט ישירות
        if loggedin_user_rank is not None:
            rank = loggedin_user_rank.lower()
            if rank == 'manager':
                admin_screen()
                return
            elif rank == 'employee':
                employee_screen()
                return

        # כותרת
        title = widgets.HTML("""
            <div style='text-align:center; margin-bottom: 20px;'>
                <h2 style='color:#2563eb; font-family:"Segoe UI Semibold", sans-serif;'>🔐 Welcome to Shablool System</h2>
                <p style='color:#6b7280;'>Please enter your username and password to log in.</p>
            </div>
        """)

        # שדות
        username_label = widgets.HTML("<span style='color:#1f2937; font-weight:bold;'>Username</span>")
        username = widgets.Text(
            placeholder='Enter your username',
            layout=widgets.Layout(width='320px')
        )

        password_label = widgets.HTML("<span style='color:#1f2937; font-weight:bold;'>Password</span>")
        password = widgets.Password(
            placeholder='Enter your password',
            layout=widgets.Layout(width='320px')
        )

        login_button = widgets.Button(
            description='Login',
            button_style='',
            layout=widgets.Layout(width='320px', height='45px', margin='20px 0')
        )

        message = widgets.HTML(value="")

        # התחברות
        def on_login_clicked(b):
            global loggedin_user
            global loggedin_user_rank
            message.value = ""

            user = username.value.strip()
            pwd = password.value.strip()

            if not user or not pwd:
                message.value = "<p style='color:#dc2626; font-weight:500;'>❌ Please enter both username and password.</p>"
                return

            user_data = FBconn.get(f'/Users/{user}', None)

            if user_data is None:
                message.value = "<p style='color:#dc2626; font-weight:500;'>❌ User does not exist.</p>"
                return

            if user_data.get('password') == pwd:
                loggedin_user = user
                loggedin_user_rank = user_data.get('rank', 'unknown')
                message.value = f"<p style='color:#16a34a; font-weight:500;'>✅ Login successful! Welcome, <b>{user}</b> ({loggedin_user_rank})</p>"
                main_dashboard()
            else:
                message.value = "<p style='color:#dc2626; font-weight:500;'>❌ Incorrect password.</p>"

        login_button.on_click(on_login_clicked)

        # טופס
        form = widgets.VBox([
            title,
            username_label,
            username,
            password_label,
            password,
            login_button,
            message
        ], layout=widgets.Layout(
            align_items='center',
            border='1px solid #e5e7eb',
            padding='30px',
            width='400px',
            border_radius='12px',
            background_color='#ffffff',
            box_shadow='0 6px 20px rgba(0,0,0,0.05)'
        ))

        # מרכז למסך
        container = widgets.HBox([form], layout=widgets.Layout(justify_content='center'))
        display(container)


In [8]:
def search_screen():
    with content_output:
        content_output.clear_output()

        # כותרת
        title = """
        <div style='text-align:center; margin-bottom: 25px;'>
            <h2 style='color:#2563eb; font-family:"Segoe UI Semibold", sans-serif;'>🔍 Shablool Search Engine</h2>
            <p style='color:#6b7280;'>Find documents by keywords or phrases</p>
        </div>
        """
        display(HTML(title))

        # שדה חיפוש
        search_input = widgets.Text(
            placeholder='Enter term...',
            description='Search:',
            layout=widgets.Layout(width='500px', height='40px'),
            style={'description_width': '80px'}
        )

        # כפתור חיפוש
        search_btn = widgets.Button(
            description="Search",
            button_style='',
            layout=widgets.Layout(width='120px', height='45px')
        )

        # תיבת תוצאות
        output = widgets.Output()

        # פונקציית חיפוש
        def do_search(b):
            query = search_input.value.strip()
            with output:
                output.clear_output()

                if not query:
                    display(HTML("<p style='color:#dc2626; font-weight:500;'>❌ Please enter a search term.</p>"))
                    return

                results = search_mqtt(query)

                if not results:
                    display(HTML(f"<p style='color:#dc2626; font-weight:500;'>❌ No results found for: <b>{query}</b></p>"))
                    return

                html = f"""
                <p style='color:#16a34a; font-weight:500;'>✅ {len(results)} result(s) for <b>{query}</b></p>
                <div style='display:flex; flex-direction:column; align-items:center; gap:12px;'>
                """

                for res in results:
                    title = res.get('title', 'Unknown Title')
                    url = res.get('url', '#')
                    matches = res.get('matching_words', 'N/A')
                    freq = res.get('total_frequency', 'N/A')

                    html += f"""
                    <div style='width:90%; max-width:600px; background:#ffffff; border-radius:10px; box-shadow:0 2px 6px rgba(0,0,0,0.05); padding:16px;'>
                        <h3 style='margin:0; font-size:16px; color:#1f2937; font-weight:600;'>{title}</h3>
                        <a href="{url}" target="_blank" style='color:#3b82f6; font-size:14px; text-decoration:underline;'>🔗 {url}</a>
                        <p style='color:#4b5563; font-size:13px; margin-top:6px;'>🧠 Matches: <b>{matches}</b> • Frequency: <b>{freq}</b></p>
                    </div>
                    """

                html += "</div>"
                display(HTML(html))

        search_btn.on_click(do_search)

        # שורת קלט וכפתור
        search_row = widgets.HBox(
            [search_input, search_btn],
            layout=widgets.Layout(
                justify_content='center',
                align_items='center',
                gap='12px',
                margin='10px 0 20px 0'
            )
        )

        # הצגה
        display(widgets.VBox([
            search_row,
            output
        ], layout=widgets.Layout(align_items='center')))


In [9]:
def stats_screen():
    with content_output:
        content_output.clear_output()

        allData = FBconn.get('/sensor_data', None)
        if not allData:
            display(HTML("<p style='color:#dc2626; font-weight:500; text-align:center;'>❌ No sensor data available.</p>"))
            return

        sources = ['indoor', 'outdoor']

        title = widgets.HTML("""
            <div style='text-align:center; margin-bottom: 30px;'>
                <h2 style='color:#2563eb; font-family:"Segoe UI Semibold", sans-serif;'>📊 Sensor Statistics Dashboard</h2>
                <p style='color:#6b7280; font-size:15px;'>Select source, sensor & chart type to visualize</p>
            </div>
        """)

        # תוויות + תיבות בחירה
        source_label = widgets.HTML("<span style='color:#1f2937; font-weight:bold;'>Source</span>")
        source_dropdown = widgets.Dropdown(
            options=sources,
            layout=widgets.Layout(width='220px')
        )

        sensor_label = widgets.HTML("<span style='color:#1f2937; font-weight:bold;'>Sensor</span>")
        sensor_dropdown = widgets.Dropdown(
            layout=widgets.Layout(width='220px')
        )

        chart_label = widgets.HTML("<span style='color:#1f2937; font-weight:bold;'>Chart Type</span>")
        chart_dropdown = widgets.Dropdown(
            options=['Line Plot', 'Histogram'],
            layout=widgets.Layout(width='220px')
        )

        # טופס בחירה מעוצב
        control_row = widgets.HBox([
            widgets.VBox([source_label, source_dropdown]),
            widgets.VBox([sensor_label, sensor_dropdown]),
            widgets.VBox([chart_label, chart_dropdown])
        ], layout=widgets.Layout(
            justify_content='center',
            flex_wrap='wrap',
            gap='24px',
            padding='20px',
            border='1px solid #e5e7eb',
            border_radius='12px',
            background_color='#ffffff',
            box_shadow='0 4px 12px rgba(0,0,0,0.05)',
            margin='0 0 30px 0'
        ))

        out = widgets.Output()

        def load_and_plot(source):
            out.clear_output()
            raw = allData.get(source, {})
            if not isinstance(raw, dict):
                raw = {}

            if not raw:
                with out:
                    display(HTML("<p style='color:#f87171; font-weight:500; text-align:center;'>❌ No data found for selected source.</p>"))
                return

            df = pd.DataFrame(raw).T
            df.index = pd.to_numeric(df.index, errors='coerce')
            df = df.loc[~df.index.isna()]
            df.index = pd.to_datetime(df.index, unit='ms')
            df = df.sort_index()

            sensors = ['Humidity', 'Pressure', 'Temperature', 'Dlight', 'Distance']
            available = [s for s in sensors if s in df.columns]
            sensor_dropdown.options = available

            if not available:
                with out:
                    display(HTML("<p style='color:#f87171; font-weight:500; text-align:center;'>❌ No valid sensor columns found.</p>"))
                return

            def redraw(change=None):
                with out:
                    out.clear_output()
                    sensor = sensor_dropdown.value
                    chart_type = chart_dropdown.value

                    try:
                        valid_mask = df[sensor].notna()
                        data = df.loc[valid_mask, sensor].astype(float)
                        timestamps = df.index[valid_mask]
                    except Exception:
                        display(HTML("<p style='color:#ef4444; font-weight:500; text-align:center;'>❌ Failed to load sensor data.</p>"))
                        return

                    fig, ax = plt.subplots(figsize=(10, 4))

                    if chart_type == 'Line Plot':
                        ax.plot(timestamps, data, marker='o', linestyle='-', color='#3b82f6')
                        ax.set_title(f"{sensor} Over Time ({source})", color='white')
                        ax.set_xlabel("Timestamp", color='white')
                        ax.set_ylabel(sensor, color='white')
                    elif chart_type == 'Histogram':
                        ax.hist(data, bins=30, color='#60a5fa', edgecolor='white')
                        ax.set_title(f"{sensor} Distribution ({source})", color='white')
                        ax.set_xlabel(sensor, color='white')
                        ax.set_ylabel("Frequency", color='white')

                    ax.tick_params(colors='white')
                    ax.grid(True, linestyle='--', alpha=0.4)
                    fig.autofmt_xdate(rotation=30)
                    fig.patch.set_facecolor('#1e2a38')
                    ax.set_facecolor('#1e2a38')
                    plt.tight_layout()
                    display(fig)
                    plt.close(fig)

            sensor_dropdown.observe(redraw, names='value')
            chart_dropdown.observe(redraw, names='value')
            redraw()

        source_dropdown.observe(lambda change: load_and_plot(change['new']), names='value')

        display(widgets.VBox([
            title,
            control_row,
            out
        ], layout=widgets.Layout(align_items='center')))

        load_and_plot('indoor')


In [10]:
def export_data_screen():
    with content_output:
        content_output.clear_output()

        data = FBconn.get('/sensor_data', None)
        if not data:
            display(HTML("<p style='color:#dc2626; font-weight:500; text-align:center;'>❌ No sensor data found.</p>"))
            return

        recent = {k: v for k, v in data.items() if k.isdigit()}
        recent_sorted = sorted(recent.items(), key=lambda x: int(x[0]), reverse=True)[:5]

        if not recent_sorted:
            display(HTML("<p style='color:#f59e0b; font-weight:500; text-align:center;'>⚠️ No recent sensor records available.</p>"))
            return

        # יצירת DataFrame
        rows = []
        for ts, entry in recent_sorted:
            time_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(ts) / 1000))
            row = {"Timestamp": time_str}
            row.update(entry)
            rows.append(row)
        df = pd.DataFrame(rows)

        # כותרת
        display(HTML("""
            <div style='text-align:center; margin-bottom:20px;'>
                <h2 style='color:#2563eb; font-family:"Segoe UI Semibold", sans-serif;'>💾 Latest Sensor Data</h2>
            </div>
        """))

        # טבלה בעיצוב מודרני ובהיר
        html_table = df.to_html(
            index=False,
            border=0,
            classes='sensor-table',
            justify='center'
        )

        styled_table = """
        <style>
            .sensor-table {
                margin: auto;
                border-collapse: collapse;
                width: 90%;
                font-size: 14px;
                box-shadow: 0 4px 12px rgba(0,0,0,0.05);
                border-radius: 8px;
                overflow: hidden;
            }

            .sensor-table th {
                background-color: #f3f4f6;
                color: #1f2937;
                padding: 12px;
                text-align: center;
                font-weight: 600;
            }

            .sensor-table td {
                background-color: #ffffff;
                color: #374151;
                padding: 10px;
                text-align: center;
            }

            .sensor-table tr:nth-child(even) td {
                background-color: #f9fafb;
            }
        </style>
        """

        display(HTML(styled_table + html_table))

        # פונקציית הורדת CSV
        def download_csv(_):
            file_path = "/tmp/latest_sensor_data.csv"
            df.to_csv(file_path, index=False)
            with open(file_path, 'r') as f:
                content = f.read()
            display(HTML(f"""
                <div style='margin-top: 25px;'>
                    <p style='color:#16a34a; font-weight:500;'>✅ CSV file ready for download:</p>
                    <pre style='background-color:#f3f4f6; color:#1f2937; padding:12px; border-radius:8px; max-height:300px; overflow:auto; font-size:13px;'>{content}</pre>
                </div>
            """))

        # כפתור הורדה
        download_button = widgets.Button(
            description="Download as CSV",
            button_style='',
            layout=widgets.Layout(width='200px', height='45px')
        )

        download_button.on_click(download_csv)

        display(widgets.HBox([download_button], layout=widgets.Layout(justify_content='center', margin='25px 0')))


In [11]:
def admin_screen():
    with content_output:
        content_output.clear_output()

    # כותרת
    display(HTML("""
        <div style='text-align: center; margin-bottom: 30px;'>
            <h2 style='color: #2563eb; font-family: "Segoe UI Semibold", sans-serif;'>🛡️ Admin Dashboard</h2>
        </div>
    """))

    # כפתורים עם גובה קבוע
    btn_Register = widgets.Button(
        description="Register a New User",
        button_style='',
        layout=widgets.Layout(width='240px', height='48px')
    )
    btn_Credit = widgets.Button(
        description="Give Extra points to user",
        button_style='',
        layout=widgets.Layout(width='240px', height='48px')
    )
    btn_leaderBoard = widgets.Button(
        description="Watch LeaderBoard",
        button_style='',
        layout=widgets.Layout(width='240px', height='48px')
    )

    # אירועים
    btn_Register.on_click(lambda b: register_screen())

    # תצוגת כפתורים
    display(widgets.HBox([btn_Register, btn_Credit, btn_leaderBoard],
                          layout=widgets.Layout(justify_content='center', margin='20px 0')))

    # טבלת משתמשים
    users = FBconn.get('/Users', None)

    if users:
        html_users = """
        <h3 style='color:#1f2937; text-align:center; margin-bottom: 20px;'>👥 Registered Users</h3>
        <table style='margin:auto; border-collapse: collapse; font-size:14px; border-radius: 10px; overflow: hidden; box-shadow: 0 4px 12px rgba(0,0,0,0.05);'>
            <thead>
                <tr style='background-color: #e5e7eb; color: #111827;'>
                    <th style='padding: 12px 24px; text-align: left;'>Username</th>
                    <th style='padding: 12px 24px; text-align: left;'>Rank</th>
                </tr>
            </thead>
            <tbody>
        """
        for i, (uname, info) in enumerate(users.items()):
            bg = '#f9fafb' if i % 2 == 0 else '#ffffff'
            rank = info.get('rank', 'N/A')
            html_users += f"""
                <tr style='background-color: {bg}; color:#1f2937;'>
                    <td style='padding: 10px 24px;'>{uname}</td>
                    <td style='padding: 10px 24px;'>{rank}</td>
                </tr>
            """
        html_users += "</tbody></table>"
        display(HTML(html_users))
    else:
        display(HTML("""
            <p style='color:#dc2626; text-align:center; font-weight:500;'>❌ No users found in the system.</p>
        """))


In [12]:
def firebase_register(username, password, rank):
    FBconn.put('/Users', username, {
        'password': password,
        'rank': rank
    })
    print(f"User '{username}' registered successfully.")


In [13]:
def register_screen():
    with content_output:
        content_output.clear_output()

        # כותרת
        html = """
        <div style='text-align: center; margin-bottom: 20px;'>
            <h2 style='color:#2563eb; font-family: "Segoe UI Semibold", sans-serif;'>📝 Register a New User</h2>
        </div>
        """
        display(HTML(html))

        # עיצוב שדות קלט
        input_layout = widgets.Layout(width='350px')
        description_style = {'description_width': '90px'}

        username = widgets.Text(
            description='Username:',
            placeholder='Enter username',
            layout=input_layout,
            style=description_style
        )
        password = widgets.Password(
            description='Password:',
            placeholder='Enter password',
            layout=input_layout,
            style=description_style
        )
        rank_dropdown = widgets.Dropdown(
            options=['Manager', 'Employee'],
            description='Rank:',
            layout=input_layout,
            style=description_style
        )

        # כפתורים
        register_button = widgets.Button(
            description='Register',
            button_style='',
            layout=widgets.Layout(width='160px', height='45px')
        )
        btn_back = widgets.Button(
            description="Back",
            button_style='',
            layout=widgets.Layout(width='160px', height='45px')
        )

        # הודעה
        message = widgets.HTML(value="")

        # אירועים
        def on_register_clicked(b):
            uname = username.value.strip()
            pwd = password.value.strip()
            rank = rank_dropdown.value

            if not uname or not pwd:
                message.value = "<span style='color:#dc2626; font-weight:500;'>❌ Username and password are required.</span>"
                return

            firebase_register(uname, pwd, rank)
            message.value = "<span style='color:#16a34a; font-weight:500;'>✅ User registered successfully.</span>"

        register_button.on_click(on_register_clicked)
        btn_back.on_click(lambda b: admin_screen())

        # תצוגת טופס
        form = widgets.VBox(
            [username, password, rank_dropdown, message],
            layout=widgets.Layout(align_items='center', margin='0 0 20px 0')
        )
        buttons = widgets.HBox(
            [register_button, btn_back],
            layout=widgets.Layout(justify_content='center', gap='20px')
        )

        display(widgets.VBox([form, buttons]))


In [14]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

logo_url = "https://t3.ftcdn.net/jpg/05/57/42/56/360_F_557425620_aUXx64KB4Bhkj9w3DyvPtJP3ClH1fixJ.jpg"
content_output = widgets.Output()

def main_dashboard():
    clear_output(wait=True)
    content_output.clear_output()

    title_html = """
    <div style="padding: 15px; background: linear-gradient(to right, #00c6ff, #0072ff); color: white; border-radius: 10px; margin-bottom: 10px;">
        <h2 style="text-align:center; font-family:Arial Black;">Team Shablool Dashboard</h2>
    </div>
    """
    display(HTML(title_html))

    def sidebar_button(text):
        btn = widgets.Button(
            description=text,
            button_style='',
            layout=widgets.Layout(width='100%', height='40px')
        )
        btn.add_class("sidebar-btn")
        return btn

    btn_user = sidebar_button("🛠️ User")
    btn_search = sidebar_button("🔍 Search")
    btn_stats = sidebar_button("📊 Stats")
    btn_bonus = sidebar_button("💾 Export")

    btn_user.on_click(lambda b: update_active(btn_user, user_screen))
    btn_search.on_click(lambda b: update_active(btn_search, search_screen))
    btn_stats.on_click(lambda b: update_active(btn_stats, stats_screen))
    btn_bonus.on_click(lambda b: update_active(btn_bonus, export_data_screen))

    buttons = [btn_user, btn_search, btn_stats, btn_bonus]

    if loggedin_user is not None:
        btn_logout = sidebar_button("👤 LogOut")
        btn_logout.on_click(lambda b: logout_user())
        buttons.append(btn_logout)

    def update_active(active_button, callback):
        for btn in buttons:
            btn.remove_class("active-btn")
        active_button.add_class("active-btn")
        content_output.clear_output()
        callback()

    sidebar_header = widgets.VBox([
        widgets.HTML(f"""
            <div style="text-align:center; margin-bottom:10px;">
                <img src="{logo_url}" style="width:80px; height:80px; border-radius:50%; box-shadow: 2px 2px 6px rgba(0,0,0,0.2);">
                <h3 style="color:#00e6e6; font-family:sans-serif; margin-top:10px;">Dashboard</h3>
            </div>
        """)
    ])

    sidebar = widgets.VBox(
        [sidebar_header] + buttons,
        layout=widgets.Layout(
            width='220px',
            padding='15px',
            border_radius='10px',
            min_height='550px'
        ),
        _dom_classes=['sidebar']
    )

    main_area = widgets.VBox(
        [content_output],
        layout=widgets.Layout(
            width='80%',
            padding='20px',
            background_color='white',
            border='1px solid #ccc',
            border_radius='10px',
            min_height='550px',
            box_shadow='2px 2px 10px rgba(0,0,0,0.1)'
        )
    )

    main_area_wrapper = widgets.HBox(
        [main_area],
        layout=widgets.Layout(
            justify_content='center',
            width='100%'
        )
    )

    layout = widgets.HBox(
        [sidebar, main_area_wrapper],
        layout=widgets.Layout(width='100%', align_items='flex-start')
    )

    if loggedin_user is not None:
        display(HTML(f"""
        <div class="welcome-banner">
            <b>Welcome, {loggedin_user}</b> <span style="color:gray;">({loggedin_user_rank})</span>
        </div>
        """))

    display(HTML("""
    <style>
        body {
            background-color: #f4f6f9;
            font-family: 'Segoe UI', sans-serif;
            font-size: 15px;
            color: #111827;
            margin: 0;
            padding: 0;
        }

        .sidebar {
            background-color: #ffffff;
            border-right: 1px solid #e5e7eb;
            padding: 30px 20px;
            border-radius: 10px;
            min-height: 600px;
            display: flex;
            flex-direction: column;
            align-items: center;
            box-shadow: 2px 0 10px rgba(0,0,0,0.03);
        }

        .sidebar-btn {
            background-color: #3b82f6 !important;
            color: white !important;
            border: none !important;
            font-weight: 500;
            font-size: 15px;
            border-radius: 8px;
            transition: all 0.3s ease;
            margin-bottom: 12px;
            padding: 10px 16px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.05);
        }

        .sidebar-btn:hover {
            background-color: #2563eb !important;
            transform: translateY(-2px);
            cursor: pointer;
        }

        .active-btn {
            background-color: #1d4ed8 !important;
            color: white !important;
            box-shadow: 0 4px 10px rgba(29, 78, 216, 0.25);
        }

        h2, h3 {
            font-family: 'Segoe UI Semibold', sans-serif;
            margin: 0;
            color: #1f2937;
        }

        .welcome-banner {
            background: #e0f2fe;
            padding: 12px 20px;
            border-radius: 8px;
            color: #0369a1;
            margin: 15px auto;
            font-size: 15px;
            font-weight: 500;
            display: inline-block;
            border: 1px solid #bae6fd;
        }

        .container-box {
            background-color: white;
            color: #1f2937;
            border-radius: 12px;
            padding: 30px;
            box-shadow: 0 4px 20px rgba(0,0,0,0.05);
        }

        img {
            transition: all 0.3s ease;
            border-radius: 8px;
        }

        img:hover {
            transform: scale(1.02);
            box-shadow: 0 4px 12px rgba(0,0,0,0.15);
        }

        .widget-button {
            font-weight: 600;
            font-size: 15px;
            line-height: 1.2;
            padding: 12px 18px;
            border-radius: 10px;
            background-color: #3b82f6 !important;
            color: white !important;
            border: none !important;
            transition: 0.3s ease;
        }

        .widget-button:hover {
            background-color: #2563eb !important;
            transform: translateY(-2px);
            cursor: pointer;
        }
    </style>
    """))

    display(layout)


In [16]:
main_dashboard()
